In [2]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import re
import collections 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Data gathering and data cleaning

In [3]:
data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/cmps242-spring18-hw5/test.csv')

In [4]:
def string_from_list(A):
    if len(A)>0:
        B = (" ".join(A)).lower()
        return re.sub('[^a-zA-z0-9\s]','',B)
    else:
        return None 

In [5]:
data['hashtags'] =  data['tweet'].apply((lambda x: re.findall(r"#(\w+)", x)))
test_data['hashtags'] =  test_data['tweet'].apply((lambda x: re.findall(r"#(\w+)", x)))

data['mention'] =  data['tweet'].apply((lambda x: re.findall(r"#(\w+)", x)))
test_data['mention'] =  test_data['tweet'].apply((lambda x: re.findall(r"#(\w+)", x)))

data['text']  = data['tweet'].apply((lambda x: re.sub(r"#(\w+)", '', x)))
data['text']  = data['text'].apply((lambda x: re.sub(r"@(\w+)", '', x)))

test_data['text']  = test_data['tweet'].apply((lambda x: re.sub(r"#(\w+)", '', x)))
test_data['text']  = test_data['text'].apply((lambda x: re.sub(r"@(\w+)", '', x)))

In [6]:
trump_tags = ["littlemarco", "makeamericagreatagain", "trump2016", "crookedhillary", "votetrumpsc", 
              "votetrump", "trumppence16", "trumppence2016", "americafirst", "votetrumpma", "neverhillary",
              "trump4president", "lightweightsenatormarcorubio", "trumpinstagram", "americagreatagain", 
              "trump4vets", "votetrumpmi", "votetrumphi", "votetrumpnh", "caucusfortrump", "trumpforpresident",
              "makeyoutubegreatagain", "votetrumpms", "votetrumpil", "votetrumpny", "votetrumpks", "trumppence2016",
             ]

trump_list   = []
clinton_list = []

for i in range(test_data.shape[0]):
    test_data['hashtags'][i] = [x.lower() for x in test_data['hashtags'][i]]
    
    for item in trump_tags:
        if item in test_data['hashtags'][i]:
            trump_list.append(i)
            break
        
    if "lovetrumpshate" in test_data['hashtags'][i] or "nevertrump" in test_data['hashtags'][i]:
        clinton_list.append(i)
        
trump_list   = set(trump_list)
clinton_list = set(clinton_list)

In [7]:
all_hashtags_trump   = []
all_hashtags_clinton = []

for i in range(data.shape[0]):
    data['hashtags'][i] = [x.lower() for x in data['hashtags'][i]]
    if data['handle'][i]=='realDonaldTrump':
        all_hashtags_trump.extend(data['hashtags'][i])
    else:
        all_hashtags_clinton.extend(data['hashtags'][i])

all_hashtags_trump_set = set(all_hashtags_trump)
all_hashtags_clinton_set = set(all_hashtags_clinton)
#print(all_hashtags_trump_set)
#print(all_hashtags_clinton_set)

In [8]:
data['handle']    = data['handle'].replace({'realDonaldTrump':1, 'HillaryClinton':0})

data['tweet']     = data['tweet'].apply((lambda x: re.sub(r"http\S+", 'urlurlurlurl', x)))
data['tweet']     = data['tweet'].apply(lambda x: x.lower())
data['tweet']     = data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['tweet']     = data['tweet'].apply((lambda x: re.sub(' +',' ',x)))
data['tweet']     = data['tweet'].apply((lambda x: re.sub(r'(\n+)(?=[A-Z])', r' ', x)))

test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(r"http\S+", 'urlurlurlurl', x)))
test_data['tweet']     = test_data['tweet'].apply(lambda x: x.lower())
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(' +',' ',x)))
test_data['tweet']     = test_data['tweet'].apply((lambda x: re.sub(r'(\n+)(?=[A-Z])', r' ', x)))


all_tweets = pd.concat([data['tweet'], test_data['tweet']])

X       = data['tweet']
Y       = np.array(data['handle'].tolist())
X_test  = test_data['tweet']

# Bag of word's model, feature selection

In [9]:
count_vec_hashtag          = CountVectorizer()
tfidf_transformer_hashtag  = TfidfTransformer()

X_hashtag       = [" ".join(x) for x in data['hashtags']]
X_test_hashtag  = [" ".join(x) for x in test_data['hashtags']]

X_counts_hashtag         = count_vec_hashtag.fit_transform(X_hashtag)
X_tfidf_hashtag          = tfidf_transformer_hashtag.fit_transform(X_counts_hashtag)

X_test_count_hashtag     = count_vec_hashtag.transform(X_test_hashtag)
X_test_tfidf_hashtag     = tfidf_transformer_hashtag.fit_transform(X_test_count_hashtag)

In [10]:
X_tfidf_hashtag_np      = X_tfidf_hashtag.toarray()
X_test_tfidf_hashtag_np = X_test_tfidf_hashtag.toarray()

X_train_tfidf_hashtag, X_val_tfidf_hashtag, Y_train_tfidf_hashtag, Y_val_tfidf_hashtag = \
                  train_test_split(X_tfidf_hashtag_np, Y, test_size = 0.1)#, random_state = 42)

bow_dim_size_hashtag    = X_tfidf_hashtag.shape[1]

In [11]:
count_vec_mention          = CountVectorizer()
tfidf_transformer_mention  = TfidfTransformer()

X_mention       = [" ".join(x) for x in data['mention']]
X_test_mention  = [" ".join(x) for x in test_data['mention']]

X_counts_mention         = count_vec_mention.fit_transform(X_mention)
X_tfidf_mention          = tfidf_transformer_mention.fit_transform(X_counts_mention)

X_test_count_mention     = count_vec_mention.transform(X_test_mention)
X_test_tfidf_mention     = tfidf_transformer_mention.fit_transform(X_test_count_mention)

In [12]:
X_tfidf_mention_np      = X_tfidf_mention.toarray()
X_test_tfidf_mention_np = X_test_tfidf_mention.toarray()
X_train_tfidf_mention, X_val_tfidf_mention, Y_train_tfidf_mention, Y_val_tfidf_mention = \
                  train_test_split(X_tfidf_mention_np, Y, test_size = 0.1)#, random_state = 42)
bow_dim_size_mention    = X_tfidf_mention.shape[1]

In [13]:
count_vec         = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_bow_pd, X_val_bow_pd, Y_train_bow_pd, Y_val_bow_pd = train_test_split(data['text'], Y,\
                                                                              test_size=0.1)#, random_state = 42)

X_train_counts    = count_vec.fit_transform(X_train_bow_pd)
X_train_tfidf     = tfidf_transformer.fit_transform(X_train_counts)

X_val_counts      = count_vec.transform(X_val_bow_pd)
X_val_tfidf       = tfidf_transformer.transform(X_val_counts)

In [14]:
#hyperVec                  = [1e6,1e5,1e4,1e3,1e2,1e1,1e0,1e-1,1e-2]
#loss_dic_bow              = {}

#for C in hyperVec:
#    lf_tmp = LogisticRegression(C=C)
#    lf_tmp.fit(X_train_tfidf, Y_train_bow_pd)
#    y_val_predicted_tmp = lf_tmp.predict(X_val_tfidf)
#    loss_dic_bow[C] = log_loss(y_pred = y_val_predicted_tmp, y_true =  Y_val_bow_pd)
   
#C_optimal = min(loss_dic_bow, key=loss_dic_bow.get)

In [15]:
X_counts           = count_vec.fit_transform(X)
X_tfidf            = tfidf_transformer.fit_transform(X_counts)

X_test_counts      = count_vec.transform(X_test)
X_test_tfidf       = tfidf_transformer.transform(X_test_counts)

#lf = LogisticRegression(C = C_optimal)
#lf.fit(X_tfidf, Y)
#Y_bow_predicted_test = lf.predict(X_test_tfidf)

#print(len(Y_bow_predicted_test))

In [16]:
X_tfidf_np      = X_tfidf.toarray()
X_test_tfidf_np = X_test_tfidf.toarray()
X_train_tfidf, X_val_tfidf, Y_train_tfidf, Y_val_tfidf = train_test_split(X_tfidf_np,\
                                                                          Y, test_size = 0.1)#, random_state = 42)
bow_dim_size    = X_tfidf.shape[1]
#print(X_test_tfidf_np.shape)
#print(X_tfidf_np.shape)

In [17]:
X_all = []
for i in range(X.shape[0]):
    X_all.append(X[i].split())
    
for i in range(X_test.shape[0]):
    X_all.append(X_test[i].split())

In [18]:
tweets         = all_tweets.tolist()
words          = " ".join(tweets)
words          = " ".join(words.split("\n")) 
words          = " ".join(words.split("\t"))
words          = " ".join(words.split("\xa0")).split()
whole_text     = " ".join(words)

count          = collections.Counter(words)
count_clean    = [(item, count[item]) for item in count if count[item]>1]
vocab_size     = len(count_clean)
num_words      = vocab_size+2 

dic = {}

for i in range(vocab_size):
    dic[count_clean[i][0]] = (i+1, count_clean[i][1])

In [19]:
def string_to_token_list(A):
    B      = A.strip().split(" ")
    output = []
    for item in B:
        if item in dic:
            output.append(dic[item][0])
        else:
            output.append(vocab_size + 1)
    return output

In [20]:
X_tokens       = X.apply(lambda x: string_to_token_list(x))
X_test_tokens  = X_test.apply(lambda x: string_to_token_list(x)) 

num_tokens     = [len(tokens) for tokens in X_tokens]
num_tokens     = np.array(num_tokens)
max_tokens     = int(np.max(num_tokens))

X_pad          = []
X_test_pad     = []

for item in X_tokens:
    n = len(item)
    X_pad.append([0]*(max_tokens-n) + item)

for item in X_test_tokens:
    n = len(item)
    X_test_pad.append([0]*(max_tokens-n) + item)
    
X_pad        = np.array(X_pad)
X_test_pad   = np.array(X_test_pad)
X_all_pad    = np.concatenate((X_pad , X_test_pad), axis=0)

In [21]:
X_train_pad, X_val_pad, Y_train_pad, Y_val_pad = train_test_split(X_pad, Y, test_size = 0.1)#, random_state = 42)

# Hyper parameter selection

In [22]:
lr                         = 1e-5

lstm_dropout_prob          = 0.5
lstm_forget_bias           = 1.0
reg_const                  = 100.0
reg_const_bow              = 10.0

embedding_size             = 64
batch_size                 = 32
lstm_out                   = 32

bow_hidden_layer_1_size    = 128
bow_hidden_layer_2_size    = 32

hashtag_hidden_layer_size  = 32
mention_hidden_layer_size  = 32

hidden_layer_1_size        = lstm_out + bow_hidden_layer_2_size + hashtag_hidden_layer_size + mention_hidden_layer_size
hidden_layer_2_size        = 64 #lstm_out + bow_hidden_layer_2_size

# Word embedding initialization 

In [23]:
from gensim.models import Word2Vec

model = Word2Vec(X_all, iter=100, min_count=2, size=embedding_size, workers=4)
#words = list(model.wv.vocab)
#print(words)
#print(model.wv.similar_by_word('makeamericagreatagain'))
#model.wv['trump'] 

In [24]:
embedding_matrix = np.zeros((1,embedding_size), np.float32)

for i in range(vocab_size):
    word             = count_clean[i][0]
    word_vec         = model.wv[word]
    embedding_matrix = np.vstack((embedding_matrix, word_vec))

embedding_matrix = np.vstack((embedding_matrix, 5* np.ones((1,embedding_size), np.float32)))

# Utility functions

In [25]:
def evaluate_test_data_set(filename, X_test_pad, X_test_tfidf_np, X_test_tfidf_hashtag_np, X_test_tfidf_mention_np, sess):
        test_size   = X_test_pad.shape[0]
        test_remain = batch_size - (test_size % batch_size) 
        X_test_pad      = np.concatenate((X_test_pad, np.zeros((test_remain, max_tokens), np.float32)), axis=0)
        X_test_tfidf_np = np.concatenate((X_test_tfidf_np, np.zeros((test_remain, bow_dim_size), np.float32)), axis=0)
        X_test_tfidf_hashtag_np = np.concatenate((X_test_tfidf_hashtag_np, \
                                                  np.zeros((test_remain, bow_dim_size_hashtag), np.float32)), axis=0)
        X_test_tfidf_mention_np = np.concatenate((X_test_tfidf_mention_np, \
                                                  np.zeros((test_remain, bow_dim_size_mention), np.float32)), axis=0)
        
        m = test_size // batch_size 
        pred_test_vals = np.empty((0, 2), np.float32)

        for i in range(m+1):
            input_test_batch              = X_test_pad[i * batch_size : (i+1) * batch_size, :]
            input_X_test_tfidf            = X_test_tfidf_np[i * batch_size : (i+1) * batch_size, :]
            input_X_test_tfidf_hashtag    = X_test_tfidf_hashtag_np[i * batch_size : (i+1) * batch_size, :]
            input_X_test_tfidf_mention    = X_test_tfidf_mention_np[i * batch_size : (i+1) * batch_size, :]
            test_preds_list  = sess.run([predictions], feed_dict = {X_bow     : input_X_test_tfidf,
                                                                    X_hashtag : input_X_test_tfidf_hashtag,
                                                                    X_mention : input_X_test_tfidf_mention,
                                                                    X         : input_test_batch})
            pred_test_batch  = np.asarray(test_preds_list).reshape(batch_size, 2)
            pred_test_vals   = np.concatenate((pred_test_vals, pred_test_batch), axis=0)
            
        with open(filename,"w+") as outputfile:
            outputfile.write("id,realDonaldTrump,HillaryClinton\n")
            for j in range(test_size):
                #if j in trump_list:
                #    hillary = 0 
                #    donald  = 1 
                #elif j in clinton_list:
                #    hillary = 1 
                #    donald  = 0
                #else:
                hillary = pred_test_vals[j][1]
                donald  = pred_test_vals[j][0]
                outputfile.write(str(j)+","+str(donald)+","+str(hillary)+"\n")

In [26]:
def get_batch(X, Y, X_tfidf, X_tfidf_hashtag, X_tfidf_mention, size):
    rdm         = np.random.choice(X.shape[0], size , replace = False)
    y1          = Y[rdm].reshape((size,1))
    y2          = (y1+1)%2
    Y_out       = np.concatenate((y1, y2), axis=1)
    X_out       = X[rdm,:]
    X_tfidf_out = X_tfidf[rdm,:] 
    X_tfidf_hashtag_out = X_tfidf_hashtag[rdm,:]
    X_tfidf_mention_out = X_tfidf_mention[rdm,:]
    return X_tfidf_mention_out, X_tfidf_hashtag_out, X_tfidf_out, X_out, Y_out

#### Look at 
https://github.com/bernhard2202/twitter-sentiment-analysis/blob/master/model/lstm.py

For a similar implementation

# Computational graph

In [27]:
with tf.device("/cpu:0"):
    
    tf.reset_default_graph()
    tf.set_random_seed(0)

    X_bow     = tf.placeholder(tf.float32, shape=[batch_size, bow_dim_size])
    X_hashtag = tf.placeholder(tf.float32, shape=[batch_size, bow_dim_size_hashtag])
    X_mention = tf.placeholder(tf.float32, shape=[batch_size, bow_dim_size_mention])
    X         = tf.placeholder(tf.int32, shape=[batch_size, max_tokens])
    Y         = tf.placeholder(tf.float32, shape=[batch_size,2])
        
    embeddings     = tf.Variable(embedding_matrix)    
    embed          = tf.nn.embedding_lookup(embeddings, X)
    embed          = tf.unstack(embed, max_tokens, 1)
    
    lstm_cell                 = tf.contrib.rnn.BasicLSTMCell(lstm_out, forget_bias = lstm_forget_bias)
    lstm_cell                 = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob=lstm_dropout_prob)
    lstm_output , lstm_state  = tf.contrib.rnn.static_rnn(lstm_cell, inputs=embed , dtype=tf.float32)
    
    outputs                   = tf.stack(lstm_output)
    outputs                   = tf.gather(outputs, max_tokens-1, axis=0)
    outputs                   = tf.reshape(outputs, [batch_size, lstm_out])
    lstm_final_output         = tf.reshape(outputs, [batch_size , lstm_out])
    
    bow_hidden_layer_1_output = tf.layers.dense(X_bow, bow_hidden_layer_1_size, 
                                               kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const_bow))
    bow_hidden_layer_2_output = tf.layers.dense(bow_hidden_layer_1_output, bow_hidden_layer_2_size, 
                                               kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const_bow))
    
    hashtag_hidden_layer      = tf.layers.dense(X_hashtag, hashtag_hidden_layer_size, 
                                               kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const_bow))
    
    mention_hidden_layer      = tf.layers.dense(X_mention, mention_hidden_layer_size, 
                                               kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const_bow))
    
    bow_concat_lstm           = tf.concat( [lstm_final_output, X_bow, X_hashtag, X_mention], axis=1)
    
    hidden_layer_1_output     = tf.layers.dense(bow_concat_lstm, hidden_layer_1_size,
                                                kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const) )
    hidden_layer_2_output     = tf.layers.dense(hidden_layer_1_output, hidden_layer_2_size,
                                                kernel_regularizer = tf.contrib.layers.l2_regularizer(reg_const))
    
    out_weight      = tf.Variable(tf.random_normal([hidden_layer_2_size, 2]))
    out_bias        = tf.Variable(tf.random_normal([2]))
    
    scores          = tf.nn.xw_plus_b(hidden_layer_2_output, out_weight,out_bias)
    predictions     = tf.nn.softmax(scores)
    
    losses          = tf.nn.softmax_cross_entropy_with_logits_v2(logits = scores,
                                                              labels = Y)
    loss            = tf.reduce_mean(losses)
    
    correct_pred    = tf.equal(tf.argmax(predictions, 1), tf.argmax(Y, 1))
    accuracy        = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    optimizer       = tf.train.AdamOptimizer(lr).minimize(loss)

# Running the computational graph

In [ ]:
epochs           = 8000
display_epoch    = 100

import time
timestr = time.strftime("%Y%m%d-%H%M%S")
allstr = "batchSize_" + str(batch_size) + "_embeddingSize_" + str(embedding_size) + \
         "_lstmOut_" + str(lstm_out) + "_bowSize2_" + str(bow_hidden_layer_2_size) + \
         "_hidden1_" + str(hidden_layer_1_size) + "_hidden2_" + str(hidden_layer_2_size) + "_time_"+ timestr
filename = "/Users/mohsenkiskani/Downloads/submissions/structred/lstm_bow_"+str(allstr)+".csv"

loss_val_vec     = []
accur_val_vec    = []
loss_train_vec   = []
accur_train_vec  = []
epoch_vec        = []

with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(epochs):
            batch_mention_vals, batch_hashtag_vals, batch_bow_vals, batch_inputs, batch_labels = get_batch(X_train_pad,
                                                                                                           Y_train_pad,
                                                                                                           X_train_tfidf,
                                                                                                           X_train_tfidf_hashtag,
                                                                                                           X_train_tfidf_mention,
                                                                                                           batch_size)
            loss_train, accur_train, _ = sess.run([loss, accuracy, optimizer],
                                                  feed_dict = {X_bow     : batch_bow_vals,
                                                               X_hashtag : batch_hashtag_vals,
                                                               X_mention : batch_mention_vals,
                                                               X         : batch_inputs,
                                                               Y         : batch_labels})
            
            batch_mention_vals, batch_hashtag_vals, batch_bow_vals, batch_inputs, batch_labels = get_batch(X_val_pad,
                                                                                                           Y_val_pad,
                                                                                                           X_val_tfidf,
                                                                                                           X_val_tfidf_hashtag,
                                                                                                           X_val_tfidf_mention,
                                                                                                           batch_size)
            
            loss_val, accur_val = sess.run([loss, accuracy],
                                           feed_dict = {X_bow     : batch_bow_vals,
                                                        X_hashtag : batch_hashtag_vals,
                                                        X_mention : batch_mention_vals,
                                                        X         : batch_inputs,
                                                        Y         : batch_labels})
                
            loss_train_vec.append(loss_train)
            accur_train_vec.append(accur_train)
            loss_val_vec.append(loss_val)
            accur_val_vec.append(accur_val)
            epoch_vec.append(epoch)
            
            if epoch % display_epoch == 0:
                print("Epoch " +str(epoch) + ", Batch loss = ", 
                      "%.3f, " % loss_train , "training accuray = ",  "%.3f, " % accur_train, "val loss = ",
                      "%.3f, " % loss_val , "val accuray = ",  "%.3f, " % accur_val)
        
        evaluate_test_data_set(filename, X_test_pad, X_test_tfidf_np, X_test_tfidf_hashtag_np, X_test_tfidf_mention_np, sess)

Epoch 0, Batch loss =  0.852,  training accuray =  0.625,  val loss =  1.115,  val accuray =  0.438, 
Epoch 100, Batch loss =  0.937,  training accuray =  0.500,  val loss =  1.118,  val accuray =  0.344, 
Epoch 200, Batch loss =  0.839,  training accuray =  0.438,  val loss =  0.810,  val accuray =  0.500, 
Epoch 300, Batch loss =  0.698,  training accuray =  0.562,  val loss =  1.032,  val accuray =  0.375, 
Epoch 400, Batch loss =  0.557,  training accuray =  0.750,  val loss =  0.821,  val accuray =  0.438, 
Epoch 500, Batch loss =  0.734,  training accuray =  0.469,  val loss =  0.650,  val accuray =  0.688, 
Epoch 600, Batch loss =  0.661,  training accuray =  0.625,  val loss =  0.651,  val accuray =  0.625, 
Epoch 700, Batch loss =  0.515,  training accuray =  0.812,  val loss =  0.640,  val accuray =  0.562, 
Epoch 800, Batch loss =  0.615,  training accuray =  0.750,  val loss =  0.600,  val accuray =  0.688, 
Epoch 900, Batch loss =  0.526,  training accuray =  0.719,  val l

# Plotting the results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plotting_parameter = 300

const_vec_02 = [0.2]* len(epoch_vec)
const_vec_01 = [0.1]* len(epoch_vec)

fig1, (ay1, ay2) = plt.subplots(2, 1)
ay1.plot(epoch_vec[plotting_parameter:], loss_train_vec[plotting_parameter:])
ay1.plot(epoch_vec[plotting_parameter:], const_vec_01[plotting_parameter:])
ay1.plot(epoch_vec[plotting_parameter:], const_vec_02[plotting_parameter:])

ay2.plot(epoch_vec[plotting_parameter:], loss_val_vec[plotting_parameter:])
ay2.plot(epoch_vec[plotting_parameter:], const_vec_01[plotting_parameter:])
ay2.plot(epoch_vec[plotting_parameter:], const_vec_02[plotting_parameter:])


fig2, (ay3, ay4) = plt.subplots(2, 1)
ay3.plot(epoch_vec[plotting_parameter:], accur_train_vec[plotting_parameter:])
#ay3.plot(epoch_vec[plotting_parameter:], const_vec[plotting_parameter:])
ay4.plot(epoch_vec[plotting_parameter:], accur_val_vec[plotting_parameter:])
#ay4.plot(epoch_vec[plotting_parameter:], const_vec[plotting_parameter:])


ay1.set_title('Training batch loss vs epoch')
ay1.set_xlabel('Epoch')
ay1.set_ylabel('Training batch loss')

ay2.set_title('Validation batch loss vs epoch')
ay2.set_xlabel('Epoch')
ay2.set_ylabel('Validation batch loss')

ay3.set_title('Training batch accuracy vs epoch')
ay3.set_xlabel('Epoch')
ay3.set_ylabel('Train accuracy')

ay4.set_title('Validation batch accuracy vs epoch')
ay4.set_xlabel('Epoch')
ay4.set_ylabel('Validation accuracy')

fig1.set_size_inches(18.5, 10.5)
fig1.savefig("/Users/mohsenkiskani/Downloads/submissions/structred/loss_"+timestr+".png", dpi=100)

fig2.set_size_inches(18.5, 10.5)
fig2.savefig("/Users/mohsenkiskani/Downloads/submissions/structred/accuracy_"+timestr+".png", dpi=100)
plt.show()